# Provided code

You shouldn't need to change anything in this section.

### Load data to Colab

In [3]:
if False:  # manual loading
    from google.colab import file
    uploaded = files.upload()  # then browse, select the files
    
else:  # automatic loading
    import requests
    import gzip
    
    filepath_d_gr = 'http://users.diag.uniroma1.it/challenge9/data/USA-road-d/' + 'USA-road-d.NY.gr.gz'
    filepath_t_gr = 'http://users.diag.uniroma1.it/challenge9/data/USA-road-t/' + 'USA-road-t.NY.gr.gz'
    filepath_d_co = 'http://users.diag.uniroma1.it/challenge9/data/USA-road-d/' + 'USA-road-d.NY.co.gz'
    
    def loader(url):
        name = url.rsplit('/', 1)[1].rsplit('.', 1)[0]
        savename = name + '.txt'
        
        with open(savename, 'wb') as f_out:
            with requests.get(url) as r:
                f_in = gzip.decompress(r.content)
                f_out.write(f_in)
                
        print(savename)
            
    loader(filepath_d_gr)
    loader(filepath_t_gr)
    loader(filepath_d_co)

USA-road-d.NY.gr.txt
USA-road-t.NY.gr.txt
USA-road-d.NY.co.txt


### Graph and Vertex classes

In [4]:
# Vertex implementation
class Vertex:
    # Initialization of a vertex, given a neighbor and the corresponding weight
    # Each vertex contains a list of neighbors and corresponding weights
    def __init__(self, i, neighbor_index, weight):
        self.index = i
        self.neighbors = [neighbor_index]
        self.weights = [weight]
        
    def getNeighbors(self):
        return self.neighbors
    
    def getWeights(self):
        return self.weights
    
    # Add a neighbor with corresponding weight to the vertex
    def _addNeighbor(self, neighbor_index, weight):
        self.neighbors.append(neighbor_index)
        self.weights.append(weight)


# Graph data structure
class Graph:
    # Initializes a graph with n_vertices nodes
    # The graph contains a list of vertices
    def __init__(self, n_vertices):
        self.vertices = [None] * (n_vertices+1)
        self.num_vertices = len(self.vertices)
    
    # Returns the i'th node
    def getVertex(self, i):
        if ((i > len(self.vertices)) | (i <= 0)):
            raise ValueError(f'index {i} is out of bounds')
        else:
            return self.vertices[i]
    
    # Adds a new vertex to the graph
    def _addVertex(self, vertex_index, neighor_index, distance):
        if (self.vertices[vertex_index] == None):
            # Construct new vertex
            self.vertices[vertex_index] = Vertex(vertex_index, neighor_index, distance)
        else:
            # Vertex already in graph but other neighbor, add extra edge
            self.vertices[vertex_index]._addNeighbor(neighor_index, distance)


In [5]:
import fileinput

# Read graph data
def readGraph(filePath):
    n_vertices = 0
    for line in fileinput.input([filePath]):
        words = line.split(" ")
        if (words[0] == "p"):
            n_vertices = int(words[2])
    graph = Graph(n_vertices)
    for line in fileinput.input([filePath]):
        words = line.split(" ")
        if (words[0] == "a"):
            graph._addVertex(int(words[1]), int(words[2]), float(words[3]))
    return graph


# Read coordinates data
def readCoordinates(filepath):
    # Start to count from 1
    coordinates = [None]
    for line in fileinput.input([filepath]):
        words = line.split(" ")
        if (words[0] == "v"):
            coordinates.append([float(words[2]), float(words[3])])
    return coordinates


### Usefull functions

In [6]:
import numpy as np
    
# Priority queue definition
class PriorityQueue(dict):
    def put(self, item, value):
        # Watch out that value is not overwritten with higher value, shouldn't be allowed to happen!
        if item in self :
            value = min( value, self[item])
        self[item] = value
    def pop(self):
        """
        Returns the item with the lowest weight
        """
        item_min = min(self, key=self.get)
        super().pop(item_min)
        return item_min

    
def angles2centimeters(lo, la):
    """
    Convert longitude and latitude to local orthogonal grid
    :param lo: longitude
    :param la: latitude
    :return: height and width coordinates in cm's
    """
    
    radius = 6300 * 1e4  # cm
    la_mean = 40794234.  # 1e-6 degree
    lo_mean = -74016939.  # 1e-6 degree
    
    w = radius * np.cos(np.radians(la / 1e6)) * np.radians((lo - lo_mean) / 1e6)
    h = radius * np.radians((la - la_mean) / 1e6)
    
    return w, h 

# Assignment

## Code skeletons

Feel free to move the following code to the relevant questions. 

Before submitting your code, make sure to execute all code fields sequentially. Notebooks that don't execute sequentially will be penalised.

## Answers

Answer the questions from the assignment and add appropriate code where relevant to the question.

In [7]:
# Question 1
graph = readGraph("USA-road-d.NY.gr.txt")
vertices_nr = graph.num_vertices

edges_nr = 0
for i in range(1, vertices_nr):
    edges_nr += len(graph.getVertex(i).getNeighbors())

print("The number of vertices is {}".format(vertices_nr))
print("The number of edges is {}".format(edges_nr))

The number of vertices is 264347
The number of edges is 733846


In [8]:
# Question 2
print("A* algorithm finds the cheapest path from a start node to the goal by keeping track of the current cheapest path from start node to node n in g(n) and searching for the best path from current node n to goal using a heuristic function h(n), e.g Euclidean distance. Adding up g(n) and h(n) results in the estimated cost of the cheapest path through node n, marked by f(n). The heuristic function should be consistent and admissible, consistent meaning that its estimate to the goal from the current node is always less than or equal to the estimate from any neighbouring node to the goal PLUS the cost of reaching that neighbor from the current node. Admissible means that the heuristic function never overestimates the actual cost of reaching the goal. \n")
print("The choice of a heuristic function is vital - if h(n) =  0, the algorithm behaves as a Dijkstra's algorithm, which always finds the shortest path. If h(n) is admissible (lower than the actual cost of reaching the goal from n), it always finds the shortest path, but might make the algorithm slower. If h(n) is perfect (estimates the cost of reaching the goal from n exactly), the algorithm becomes very fast. If h(n) is not admissible (overestimates the cost sometimes), it might not find the shortest path, but is quicker.")

A* algorithm finds the cheapest path from a start node to the goal by keeping track of the current cheapest path from start node to node n in g(n) and searching for the best path from current node n to goal using a heuristic function h(n), e.g Euclidean distance. Adding up g(n) and h(n) results in the estimated cost of the cheapest path through node n, marked by f(n). The heuristic function should be consistent and admissible, consistent meaning that its estimate to the goal from the current node is always less than or equal to the estimate from any neighbouring node to the goal PLUS the cost of reaching that neighbor from the current node. Admissible means that the heuristic function never overestimates the actual cost of reaching the goal. 

The choice of a heuristic function is vital - if h(n) =  0, the algorithm behaves as a Dijkstra's algorithm, which always finds the shortest path. If h(n) is admissible (lower than the actual cost of reaching the goal from n), it always finds the

In [74]:
# Question 3
import math 

# The graph and coordinates data
# TODO: implement
graph = readGraph('USA-road-t.NY.gr.txt')
co = readCoordinates('USA-road-d.NY.co.txt')


# Heuristic function
def h(node1, node2):
    """
    Heuristic function 1
    """
    co1 = co[node1]
    co2 = co[node2]
    
    w1, l1 = angles2centimeters(co1[0], co1[1])
    w2, l2 = angles2centimeters(co2[0], co2[1])
    
    EuclidDistSq = pow( w1 - w2, 2 ) + pow( l1 - l2, 2 )
    return math.sqrt( EuclidDistSq )


def printPath( cameFrom, start, goal ) :
    node = goal
    path = [goal]
    cost = 0
    while not (node == start) :
        previous = cameFrom[node]
        weights_prev = graph.getVertex(previous).getWeights()
        index =  graph.getVertex(previous).getNeighbors().index(node)
        cost += weights_prev[index]
        node = previous
        path.append(node)
    path.reverse()
    print(path)
    return (path, cost)


# Algorithm
def a_star_search(graph, co, start, goal):
    """
    A* algorithm
    :param graph: Graph object
    :param co: coordinates list
    :param start: index of start node
    :param goal: index of goal node
    :return: The path of nodes and total length
    """
    openSet = set([start]) # A set of nodes
    closed = set([]) # A set of nodes

    cameFrom = {} # A node-node key-value mapping

    gScore = {} # A node-int key-value mapping
    gScore[start] = 0

    fScore = {} # A node-int key-value mapping
    """ CHANGE HEURISTIC FUNCTION HERE """
    #fScore[start] = gScore[start] + h(start, goal)
    timeA = time.time()
    M = h_8_minimalWeight(graph)
    pathlength = h_8_pathlengths('USA-road-t.NY.gr.txt', goal)
    fScore[start] = gScore[start] + h_8(start, M, pathlength)
    timeB = time.time()
    print(round(timeB-timeA, 2))

    nodeOrder = PriorityQueue()
    nodeOrder.put(start, fScore.get(start))

    while not len(openSet) == 0 :
        currentNode = nodeOrder.pop()
        if currentNode == goal :
            return printPath(cameFrom, start, goal)

        openSet.remove(currentNode)
        closed.add(currentNode)
        neighbours = graph.getVertex(currentNode).getNeighbors()

        weights = graph.getVertex(currentNode).getWeights()
        
        for neighbour in neighbours :
            
            if neighbour in closed:
                continue
            tentative_g_score = gScore[currentNode] + weights[ neighbours.index(neighbour) ]
            if neighbour not in openSet:
                openSet.add(neighbour)
            elif tentative_g_score >= gScore[neighbour]:
                continue
            cameFrom[neighbour] = currentNode
            gScore[neighbour] = tentative_g_score
            """ CHANGE HEURISTIC FUNCTION HERE """
            #fScore[neighbour] = gScore[neighbour] + h(neighbour, goal)
            fScore[neighbour] = gScore[neighbour] + h_8(neighbour, M, pathlength)
            nodeOrder.put(neighbour, fScore[neighbour])
            
            # consistency check #
            """ CHANGE HEURISTIC FUNCTION HERE x2 """
            #if not h(currentNode, goal) <= (weights[ neighbours.index(neighbour) ] + h(neighbour, goal)):
            if not h_8(currentNode, M, pathlength) <= (weights[ neighbours.index(neighbour) ] + h_8(neighbour, M, pathlength)):
                print("Heuristics not consistent, estimate from " + str(currentNode) + " is bigger than from " + str(neighbour))
            
    return [], 0

In [10]:
# Question 3
print("A* function with Euclidean distance heuristics does return the shortest distance path.")
print("An interesting thing we found with implementing a consistency check is that there are some nodes (e.g try the algorithm with group nr 44) for which the heuristic function is not consistent, but the algorithm does not include these nodes in the final path.")

A* function with Euclidean distance heuristics does return the shortest distance path.
An interesting thing we found with implementing a consistency check is that there are some nodes (e.g try the algorithm with group nr 44) for which the heuristic function is not consistent, but the algorithm does not include these nodes in the final path.


In [54]:
# Question 4
def h4(node1, node2):
    return 0
print("A* search with an heuristic function of h(n)=0 is optimal, as the heuristic function is consistent, because the estimate to the goal from any node is always 0, which in itself is less than the cost of reaching any following node. \nThe running time of the algorithm is long, so the algorithm with no heuristics is not time- or space-efficient.")

A* search with an heuristic function of h(n)=0 is optimal, as the heuristic function is consistent, because the estimate to the goal from any node is always 0, which in itself is less than the cost of reaching any following node. 
The running time of the algorithm is long, so the algorithm with no heuristics is not time- or space-efficient.


In [65]:
# Question 5
def h5(node1, node2):
    """
    Heuristic function 3
    """
    co1 = co[node1]
    co2 = co[node2]
    
    w1, l1 = angles2centimeters(co1[0], co1[1])
    w2, l2 = angles2centimeters(co2[0], co2[1])
    
    ManhattanDist = abs(w1-w2) +  abs(l1-l2)
    return ManhattanDist
print("Algorithm with Manhattan distance is not optimal, as the heuristic estimate is bigger from some nodes than the cost of reaching a certain following node plus the estimate to the goal from there (see consistency check when running the algorithm).\nAlgorithm with Euclidan heuristics returns the shortest distance path, but runs longer than the algorithm with Manhattan heuristics. Algorithm with Manhattan heuristics does not return the shortest distance path, but the running time in this case is very short.")

Algorithm with Manhattan distance is not optimal, as the heuristic estimate is bigger from some nodes than the cost of reaching a certain following node plus the estimate to the goal from there (see consistency check when running the algorithm).
Algorithm with Euclidan heuristics returns the shortest distance path, but runs longer than the algorithm with Manhattan heuristics. Algorithm with Manhattan heuristics does not return the shortest distance path, but the running time in this case is very short.


In [13]:
# Queston 6
def h6(node1, node2):
    return math.floor(h(node1, node2))

    '''
    co1 = co[node1]
    co2 = co[node2]
    
    w1, l1 = angles2centimeters(co1[0], co1[1])
    w2, l2 = angles2centimeters(co2[0], co2[1])
    
    EuclidDistSq = pow( w1 - w2, 2 ) + pow( l1 - l2, 2 )
    return math.floor(math.sqrt( EuclidDistSq ))
    '''
print("Consider the Euclidean heuristic, but round down the final result. It is easy to verify that this is still an admissible heuristic, as it only further underestimates the final cost, already underestimated by the Euclidean heuristic. It is also non-consistent, as the heuristic is no longer consistent. This due to the fact that two neighbouring nodes can now have the some value for the heuristic, while the cost of travelling between them is non-zero.")

Consider the Euclidean heuristic, but round down the final result. It is easy to verify that this is still an admissible heuristic, as it only further underestimates the final cost, already underestimated by the Euclidean heuristic. It is also non-consistent, as the heuristic is no longer consistent. This due to the fact that two neighbouring nodes can now have the some value for the heuristic, while the cost of travelling between them is non-zero.


In [14]:
# Question 7
## graph = readGraph('USA-road-t.NY.gr.txt')
## co = readCoordinates('USA-road-d.NY.co.txt')
# TODO complete below
print("When used without taking the now-changed weigths of the edges into account, the Euclidean and Manhatten heuristic are no longer guaranteed to be admissible. As a result, the optimality of the solution is no longer guaranteed. \nAs can be seen below, the results from the Manhatten distance are considerably worse than A* without heuristic. The Euclidean heuristic arrives at the correct result, but takes longer than without heuristic.")
print("Euclidean: cost of path: 957578.0, length of path: 632, time elapsed: 7.35")
print("Manhatten: cost of path: 982458.0, length of path: 667, time elapsed: 3.05")
print("No heuristic: cost of path: 957578.0, length of path: 632, time elapsed: 6.28")


When used without taking the now-changed weigths of the edges into account, the Euclidean and Manhatten heuristic are no longer guaranteed to be admissible. As a result, the optimality of the solution is no longer guaranteed.


In [41]:
# Question 8
## IDEA: A heuristic is -- or can be -- a solution to a simplified version of the original problem.
## Consider the same shortest-path problem, but with all edges with weight 1.

## The heuristic is:
##      - In the original problem, consider the minimal cost of an edge, say M.
##      - Say n is the length (in edges) of the shortest path start-->goal and i is the shortest path start-->node to a given node
##      - The remaining distance in edges from a given node to goal is at least (n-i).
##      - The shortest possible distance from the given node to the goal is then given by M*(n-i).

# Determine the minimal edge weight
def h_8_minimalWeight(graph) :
    minWeight = float('inf')
    vertices = [i for i in graph.vertices if i ]
    for vertex in vertices :
        for weight in vertex.getWeights() :
            minWeight = min(weight, minWeight)
    return minWeight

## Approach: reverse all edges of graph and label all nodes based on distance in edges to the goal.
#-- Read graph data, inverted --#
def readGraph_Inverted(filePath):
    n_vertices = 0
    for line in fileinput.input([filePath]):
        words = line.split(" ")
        if (words[0] == "p"):
            n_vertices = int(words[2])
    graph = Graph(n_vertices)
    for line in fileinput.input([filePath]):
        words = line.split(" ")
        if (words[0] == "a"):
            graph._addVertex(int(words[2]), int(words[1]), float(words[3]))
    return graph

# Determine the path length (in edges) from a given node to the goal
def h_8_pathlengths(filePath, goal) :
    graph_inv = readGraph(filePath)

    pathlength = [float('inf')]*graph_inv.num_vertices #  list with pathlength i from node to goal
    pathlength[goal] = 0

    openSet = set([goal]) # A set of nodes
    closed = set([]) # A set of nodes
    
    nodeOrder = PriorityQueue()
    nodeOrder.put(goal, 0)

    while not len(openSet) == 0 :
        currentNode = nodeOrder.pop()
        
        openSet.remove(currentNode)
        closed.add(currentNode)

        neighbours = graph_inv.getVertex(currentNode).getNeighbors()        
        for neighbour in neighbours :
            
            if pathlength[neighbour] > pathlength[currentNode] + 1 :
                pathlength[neighbour] = pathlength[currentNode] + 1

                openSet.add(neighbour)
                nodeOrder.put(neighbour, 0)
    
    return pathlength

def h_8(node, M, pathlength):
    """
    Heuristic function
    """
    i = pathlength[node]
    return i*M


In [73]:
# Question 8
print("Custom heuristic: cost of path: 957578.0, length of path: 632, time elapsed: 12.32")
print("The results of this heuristic (for this random seed) match those of A* algorithm without heuristic, which we take as an indication that it is a sound heuristic. It does, however, take up a considerable amount of time. \nAlthough very little effort has been put into optimisation of the heuristic, deeper analysis shows that 5.93 of the 12.32 seconds was needed for the preprocessing of the heuristic, meaning the preparation time far outweighs the gained time in this case.")

Custom heuristic: cost of path: 957578.0, length of path: 632, time elapsed: 12.32
The results of this heuristic (for this random seed) match those of A* algorithm without heuristic, which we take as an indication that it is a sound heuristic. It does, however, take up a considerable amount of time. 
Although very little effort has been put into optimisation of the heuristic, deeper analysis shows that 5.93 of the 12.32 seconds was needed for the preprocessing of the heuristic, meaning the preparation time far outweighs the gained time in this case.


In [75]:
# Calculate the path between your start and goal node. 
# Did you get the shortest-distance path? You can 
# verify your results in the distances.txt file.

import random
import time

group_number = 44 # TODO: change to your group number
num_vertices = graph.num_vertices  # TODO: number of vertices in the graph

random.seed(group_number)

start = random.randint(1, num_vertices + 1)
goal = random.randint(1, num_vertices + 1)

# Calculating the path between nodes
print( 'start : ' +  str(start) )
print( 'goal : ' + str(goal) )

time0 = time.time()
path, path_cost = a_star_search(graph, co, start, goal)
time1 = time.time()

print('cost of path: ' + str(path_cost))
path_length = len(path)
print('length of path: ' + str(path_length))
print("time elapsed: " + str(round(time1-time0, 2)))

start : 214191
goal : 61162
2.0
5.95
[214191, 214190, 214189, 214188, 214180, 214178, 214177, 214141, 214140, 214137, 214131, 214130, 214128, 214127, 213988, 213990, 213984, 213975, 213974, 213972, 213971, 213916, 213915, 213944, 213912, 213911, 213910, 213908, 213907, 213888, 213881, 213880, 213877, 213876, 205559, 205558, 209895, 202260, 198935, 198934, 209891, 209890, 205428, 205427, 209869, 209754, 209753, 209744, 209735, 209734, 209733, 209730, 209698, 201617, 201616, 209708, 209685, 209061, 201568, 201567, 209029, 209024, 208999, 208998, 209012, 208911, 208913, 208912, 214358, 208905, 208901, 208860, 208858, 208850, 208851, 208849, 206493, 206420, 206419, 206438, 206494, 206441, 206426, 206235, 206227, 206204, 214329, 205549, 214886, 205551, 205480, 205471, 205470, 205306, 205238, 214410, 214409, 203055, 203051, 214820, 214817, 203017, 203015, 214842, 214960, 203080, 203079, 204544, 202988, 202986, 202989, 202981, 202977, 202976, 202971, 202935, 202925, 202456, 202473, 214312, 21